# 10.2 Sorting Algorithms

the standard implementation of sorting in most Python implementations runs in roughly $O(n*log(n))$ time, where n is the length of the list.

In most cases, the right thing to do is to use with Python

* 1  method **list.sort** : takes a list as its first argument and **modifies** that list,sorts the list (ascending sort),
    
    
* 2 function **sorted** : takes an iterable object (e.g., a list or a dictionary) as its first argument and returns a **new** sorted list, <b>does not mutate L</b>

In [ ]:
L=[1,34,5,7,90,2]
L.sort()
print('sorted L',L)
L.reverse()
print('reversed L',L)

In [ ]:
L =[1,34,5,7,90,2]
L1 = L[ : ]
# descending sorts
L1.sort(reverse = True)
print(L)
print(L1)

In [ ]:
L=[1,34,5,7,90,2]
# returns a new list with same elements as L
L1=sorted(L)
print(L)
print(L1)

In [ ]:
L=[1,34,5,7,90,2]
L1=sorted(L,reverse = True)
print(L)
print(L1)

Present sorting algorithms here primarily to provide some practice in **thinking about algorithm design and complexity analysis**

We begin with a simple but inefficient algorithm, **selection sort**

### Selection sort 

**Selection sort** works by maintaining the **loop invariant** that, given a partitioning of the list into 

* <b>a prefix (L[0:i])</b> 

* <b>a suffix (L[i+1:len(L)])</b>,

the prefix is sorted and no element in the prefix is larger than <b>the smallest element</b> in the suffix.

We use induction to reason about **loop invariants**.

* **Base case**: At the **start** of the first iteration, the **prefix** is **empty**, i.e., the
**suffix** is the **entire list**. The invariant is  true.


* **Induction step**: At each step of the algorithm, we move one element **from the suffix to the prefix.** 


>   * We do this by appending a **minimum** element of the suffix to the end of the prefix. Because the invariant held before we moved the element,


>   * we know that after we append the element the prefix is **still sorted**. 


>   * We also know that since we removed the smallest element in the suffix, **no element in the prefix is larger than the smallest element in the suffix**.


* When **the loop is exited**, the **prefix** includes the **entire list**, and the **suffix**
is **empty**.


Therefore, the entire list is now sorted in **ascending** order.

Example:
```c
int a[8] = {8, 4, 5, 3, 2, 9, 4, 1};
```
![](./img/selSort.jpg)

In [ ]:
#Page 132, Figure 10.3
def selSort(L):
    """Assumes that L is a list of elements that can be
         compared using >.
       Sorts L in ascending order"""
    suffixStart = 0
    while suffixStart != len(L):
        #look at each element in suffix
        for i in range(suffixStart, len(L)):
            if L[i] < L[suffixStart]:
                #swap position of elements
                L[suffixStart], L[i] = L[i], L[suffixStart]
        suffixStart += 1

In [ ]:
L=[1,34,5,7,90,2]
selSort(L)
print(L)

Unfortunately, it is rather inefficient

* The complexity of the inner loop is $O(len(L))$ :

```python
   for i in range(suffixStart, len(L)):
```

* The complexity of the outer loop is also $O(len(L))$ :
```python
while suffixStart != len(L):
```

The complexity of the entire function is $O(len(L)^2)$. I.e., it is **quadratic** in the length of L.

#### SelectionSort in C

In [ ]:
%%file ./code/ds/SelectionSort.c

/* Sorting an array using Selection Sort (SelectionSort.c) */

#include <stdio.h>
#include <stdlib.h> 

void selectionSort(int a[], int size);
void print(const int a[], int iMin, int iMax);

// Sort the given array of size using selection sort
void selectionSort(int a[], int size) {
   int temp; // for swaping
   for (int i = 0; i < size - 1; ++i) {
      // for tracing
      print(a, 0, i - 1);
      print(a, i, size - 1);
 
      // [0, i-1] already sort
      // Search for the smallest element in [i, size-1]
      //  and swap with a[i]
      int minIndex = i;  // assume fist element is the smallest
      for (int j = i + 1; j < size; ++j) {
         if (a[j] < a[minIndex]) minIndex = j;
      }
      if (minIndex != i) {  // swap
         temp = a[i];
         a[i] = a[minIndex];
         a[minIndex] = temp;
      }
 
      // for tracing
      printf("=> ");
      print(a, 0, i - 1);
      print(a, i, size - 1);
      printf("\n");
   }
}
 
// Print the contents of the array in [iMin, iMax]
void print(const int a[], int iMin, int iMax) {
   printf("{");
   for (int i = iMin; i <= iMax; ++i) {
      printf("%d",a[i]);
      if (i < iMax)  printf(",");
   }
   printf("}");
}
 
int main() {
   const int SIZE = 8;
   int a[8] = {8, 4, 5, 3, 2, 9, 4, 1};
   print(a, 0, SIZE - 1);
   printf("\n"); 
   selectionSort(a, SIZE);
   print(a, 0, SIZE - 1);
   printf("\n"); 
  
   return 0;
}


In [ ]:
!gcc -o ./code/ds/SelectionSort.exe ./code/ds/SelectionSort.c

In [ ]:
!.\code\ds\SelectionSort.exe

### 10.2.1 Merge Sort

Merge sort is a prototypical <b>divide-and-conquer algorithm</b>. It was invented in 1945, by John von Neumann, and is still widely used.

In general, a divide-and-conquer algorithm is characterized by

* 1 A **threshold input size**, below which the problem is **not subdivided**

* 2 The size and number of **sub-instances** into which an instance is **split**,

* 3 The algorithm used to **combine** sub-solutions.


The threshold is sometimes called the **recursive base**. For item 2 it is usual to consider the ratio of initial problem size to sub-instance size. In most of the examples we’ve seen so far, the ratio was 2.

Like many divide-and-conquer algorithms it is most easily described recursively:

* If the list is of **length 0 or 1**, it is **already** sorted.  
 
     A **threshold input size**
     

* If the list has more than one element, **split the list** into two lists, and use **merge sort** to sort each of them.

    The size and number of **sub-instances** into which an instance is **split**,



* **Merge** the results.

   The algorithm used to **combine** sub-solutions.


<font color='blue'>**The idea**</font> is: 

>look at the **first** element of each list, and move the **smaller of the two** to the **end** of the result list.
>
>When one of the lists is empty, all that remains is to copy the remaining items from the other list.

Consider, for example, merging the two lists
```
 [1,5,12,18,19,20]
 [2,3,4,17]
```
<img src="./img/mergesort.PNG"/>

Example
```c
 int a1[8] = {8, 4, 5, 3, 2, 9, 4, 1};
```
![](./img/mergeSort.jpg)

### What is the complexity of the merge process? 

It involves two constant-time operations： 

* 1 comparing the values of elements 

* 2 copying elements from one list to another. 

The number of comparisons is **O(len(L))**, where L is the **longer** of the two lists. 

The number of copy operations is **O(len(L1) + len(L2))**, because each element gets copied exactly once. 

Therefore, merging two sorted lists is linear in **the length of the lists**

In [ ]:
#Page 134, Figure 10.4
def merge(left, right, compare):
    """Assumes left and right are sorted lists and
         compare defines an ordering on the elements.
       Returns a new sorted (by compare) list containing the
         same elements as (left + right) would contain."""
    
    result = []
    i,j = 0, 0
    while i < len(left) and j < len(right):
        if compare(left[i], right[j]):
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    while (i < len(left)):
        result.append(left[i])
        i += 1
    while (j < len(right)):
        result.append(right[j])
        j += 1
    return result

import operator
#  compare = operator.lt
def mergeSort(L, compare = operator.lt):
    """Assumes L is a list, compare defines an ordering
         on elements of L
       Returns a new sorted list containing the same elements as L"""
    if len(L) < 2:
        return L[:]
    else:
        middle = len(L)//2
        left = mergeSort(L[:middle], compare)
        right = mergeSort(L[middle:], compare)
        return merge(left, right, compare)

In [ ]:
L=[1,5,12,18,19,20,2,3,4,17]
L1=mergeSort(L,operator.gt)
print(L1)

#### Let’s analyze the complexity of mergeSort. 

We already know that the time complexity of merge is $O(len(L))$. At each level of recursion the total number of elements to be merged is $len(L)$. 

Therefore, the time complexity of mergeSort is

* $O(len(L))$ multiplied by the number of levels of recursion. 

Since mergeSort divides the list <b>in half</b> each time, we know that the number of levels of recursion is $O(log(len(L))$. 
  
Therefore, the time complexity of mergeSort is $O(n*log(n))</b>$, where n is len(L).
                                                                                                             
This improvement in time complexity comes with a price. 

<b>Selection sort</b> is an example of an <b>in-place</b> sorting algorithm. Because it works by swapping the place of elements within the list, it uses only <b>a constant amount of extra storage</b> (one element in our implementation). 

In contrast, the <b>merge sort</b> algorithm  involves making<b> copies of the list</b>. This means that its space complexity is $O(len(L))$.                                                                                                             

**A merge sort** works as follows:

* Divide the unsorted list into n sublists, each containing 1 element (a list of 1 element is considered sorted).

* Repeatedly merge sublists to produce new sorted sublists until there is only 1 sublist remaining. This will be the sorted list.

### 10.2.2 <font color="blue">Exploiting Functions</font> as Parameters

Suppose we want to sort a list of names written as firstName lastName, e.g.,

the list
```python
['Chris Terman', 'Tom Brady', 'Eric Grimson', 'Gisele Bundchen'].
```
Figure 10.5 defines two ordering functions, and then uses these to sort a list in two different ways.
```python
def mergeSort(L, compare = operator.lt):
```

In [ ]:
#Page 135, Figure 10.5
def lastNameFirstName(name1, name2):
    name1 = name1.split(' ')
    name2 = name2.split(' ')
    if name1[1] != name2[1]:
        return name1[1] < name2[1]
    else: #last names the same, sort by first name
        return name1[0] < name2[0]

def firstNameLastName(name1, name2):
    # If that argument is omitted 
    #  the string is split using arbitrary strings of whitespace characters
    #  space, tab, newline, return, and formfeed).
    name1 = name1.split()
    name2 = name2.split()
    if name1[0] != name2[0]:
        return name1[0] < name2[0]
    else: #first names the same, sort by last name
        return name1[1] < name2[1]


In [ ]:
L = ['Chris Terman', 'Tom Brady', 'Eric Grimson', 'Gisele Bundchen']
newL = mergeSort(L, lastNameFirstName)
print('Sorted by last name =', newL)
newL = mergeSort(L, firstNameLastName)
print('Sorted by first name =', newL)

### MergeSort in C

In [ ]:
%%file ./code/ds/MergeSort.c

/* Sorting an array using Merge Sort (MergeSort.c) */
#include <stdio.h>
#include <stdlib.h>
 
void mSort(int a[], int size);
void mergeSort(int a[], int iLeft, int iRight, int work[]);
void merge(int a[], int iLeftHalfLeft, int iLeftHalfRight,
           int iRightHalfLeft, int iRightHalfRight, int work[]);
void print(const int a[], int iLeft, int iRight);

 
// Sort the given array of size
void mSort(int a[], int size) {
   int work[size];  // work space
   mergeSort(a, 0, size - 1, work);
}
 
// Sort the given array in [iLeft, iRight]
void mergeSort(int a[], int iLeft, int iRight, int work[]) {
   if ((iRight - iLeft) >= 1) {   // more than 1 elements, divide and sort
      // Divide into left and right half
      int iLeftHalfLeft = iLeft;
      int iLeftHalfRight = (iRight + iLeft) / 2;   // truncate
      int iRightHalfLeft = iLeftHalfRight + 1;
      int iRightHalfRight = iRight;
 
      // Recursively sort each half
      mergeSort(a, iLeftHalfLeft, iLeftHalfRight, work);
      mergeSort(a, iRightHalfLeft, iRightHalfRight, work);
 
      // Merge two halves
      merge(a, iLeftHalfLeft, iLeftHalfRight, iRightHalfLeft, iRightHalfRight, work);
   }
}
 
// Merge two halves in [iLeftHalfLeft, iLeftHalfRight] and [iRightHalfLeft, iRightHalfRight]
// Assume that iLeftHalfRight + 1 = iRightHalfLeft
void merge(int a[], int iLeftHalfLeft, int iLeftHalfRight,
           int iRightHalfLeft, int iRightHalfRight, int work[]) {
   int size = iRightHalfRight - iLeftHalfLeft + 1;
   int iResult = 0;
   int iLeft = iLeftHalfLeft;
   int iRight = iRightHalfLeft;
   while (iLeft <= iLeftHalfRight && iRight <= iRightHalfRight) {
      if (a[iLeft] <= a[iRight]) {
         work[iResult++] = a[iLeft++];
      } else {
         work[iResult++] = a[iRight++];
      }
   }
   // Copy the remaining left or right into work
   while (iLeft <= iLeftHalfRight) work[iResult++] = a[iLeft++];
   while (iRight <= iRightHalfRight) work[iResult++] = a[iRight++];
 
   // for tracing
   print(a, iLeftHalfLeft, iLeftHalfRight);
   print(a, iRightHalfLeft, iRightHalfRight);
   printf("=> ");
   print(work, 0, size - 1);
   printf("\n");
 
   // Copy the work back to the original array
   for (iResult = 0, iLeft = iLeftHalfLeft; iResult < size; ++iResult, ++iLeft) {
      a[iLeft] = work[iResult];
   }
}
 
// Print the contents of the given array from iLeft to iRight (inclusive)
void print(const int a[], int iLeft, int iRight) {
   printf("{");
   for (int i = iLeft; i <= iRight; ++i) {
      printf("%d", a[i]);
      if (i < iRight) printf(",");
   }
   printf("} ");
}

 
int main() {
   // Test 1
   const int SIZE_1 = 8;
   int a1[8] = {8, 4, 5, 3, 2, 9, 4, 1};
 
   print(a1, 0, SIZE_1 - 1);
   printf("\n");
   mSort(a1, SIZE_1);
   print(a1, 0, SIZE_1 - 1);
   printf("\n \n");
 
   // Test 2
   const int SIZE_2 = 13;
   int a2[13] = {8, 4, 5, 3, 2, 9, 4, 1, 9, 1, 2, 4, 5};
 
   print(a2, 0, SIZE_2 - 1);
   printf("\n");
   mSort(a2, SIZE_2);
   print(a2, 0, SIZE_2 - 1);
   printf("\n \n");
    
   return 0;
}

In [ ]:
!gcc -o ./code/ds/MergeSort.exe ./code/ds/MergeSort.c

In [ ]:
!.\code\ds\MergeSort.exe

### The GNU C Library: Array Sort Function

http://www.gnu.org/software/libc/manual/html_node/Array-Sort-Function.html

To sort an array using an arbitrary comparison function, use the **qsort** function. The prototype for this function is in **stdlib.h**. 

Function:

```c
void qsort (void *array, size_t count, size_t size, comparison_fn_t compare)
```
To do this, you supply **a comparison function** to compare two elements of the array

This type is a GNU extension
```c
int comparison_fn_t (const void *, const void *);
```

In [ ]:
%%file ./code/ds/gccqsort.c

/* Sorting an array using The GNU C Library ：Sort (gccqsort.c) */

#include <stdio.h>
#include <stdlib.h> 

int compare_ints (const void * a, const void * b)  
{  
  return ( *(int*)a - *(int*)b );  
}  

int main() {
   const int SIZE = 8;
   int a[8] = {8, 4, 5, 3, 2, 9, 4, 1};
   qsort(a, SIZE, sizeof(int),compare_ints);
   
   for(int i = 0; i <SIZE; i++) 
      printf("%d ", a[i]);
   
  return 0;
}
    

In [ ]:
!gcc -o ./code/ds/gccqsort.exe ./code/ds/gccqsort.c

In [ ]:
!.\code\ds\gccqsort.exe

### 10.2.3 Sorting in Python

The sorting algorithm used in most Python implementations is called 

* <b>Timsort</b> ： https://en.wikipedia.org/wiki/Timsort

The **key idea** is to take **advantage** of the fact that in a lot of data sets the data is <b>already partially sorted</b>. 

**Timsort**’s worst-case performance is the same as merge sort’s, but on average it performs considerably better.

The Python

* 1  method **list.sort** takes a list as its first argument and **modifies** that list
    
    
* 2 function **sorted** takes an iterable object (e.g., a list or a dictionary) as its first argument and returns a **new** sorted list.

**sort(*, key=None, reverse=None)**

* This method sorts the list **in place**, using only < comparisons between items. Exceptions are not suppressed - if any comparison operations fail, the entire sort operation will fail (and the list will likely be left in a partially modified state).


* list.sort() accepts two arguments that can only be passed by keyword (keyword-only arguments):


* **key** specifies a function of one argument that is used to extract a comparison key from each list element (for example, key=str.lower). The key corresponding to each item in the list is calculated once and then used for the entire sorting process. The default value of None means that list items are sorted directly without calculating a separate key value.


* **reverse** is a boolean value. If set to True, then the list elements are sorted as if each comparison were reversed.


**sorted(iterable[, key][, reverse])**

* Return a **new** sorted list from the items in iterable.


* Has two optional arguments which must be specified as keyword arguments.


* **key** specifies a function of one argument that is used to extract a comparison key from each list element: key=str.lower. The default value is None (compare the elements directly).


* **reverse** is a boolean value. If set to True, then the list elements are sorted as if each comparison were reversed.

In [ ]:
# sorted list: new list
L = [3,5,2]
print('sorted L(a new list)=',sorted(L))
print('L=',L)

# sorted dict
# dict:iterable
D = {'a':12, 'c':5, 'b':'dog'}
print('sorted D(a new list)=',sorted(D))

# list.sort in place
L.sort()
print('L(modified L)=',L)


# but :'dict' object has no attribute 'sort'
D.sort()

Both the **list.sort** method and the **sorted** function can have two additional parameters.

>The <b>key</b> parameter plays the same role as compare in our implementation of merge sort: it is used to <b>supply the comparison function</b> to be
used.


>The <b>reverse</b> parameter specifies whether the list is to be sorted in <b>ascending or descending order</b>.

In [ ]:
L = [[1,2,3], (3,2,1,0), 'abc']
print(sorted(L, key = len, reverse = True))

In [ ]:
def getlastNameFirstName(name):
    name = name.split(' ')
    return name[1]

def getfirstNameLastName(name):
    name = name.split()
    return name[0]

In [ ]:
L = ['Chris Terman', 'Tom Brady', 'Eric Grimson', 'Gisele Bundchen']
newL = sorted(L, key=getlastNameFirstName)
print('Sorted by last name =', newL)
newL =sorted(L, key=getfirstNameLastName)
print('Sorted by first name =', newL)

#### stable sorts

Both the **list.sort** method and the sorted function provide <b>stable sorts</b>. 

>This means that if **two elements are equal** with respect to the comparison used **in the sort** 

>their **relative ordering** in the original list (or other iterable object) is **preserved** in the final list.

## Further Reading

###  8.6. bisect — Array bisection algorithm

https://docs.python.org/3.5/library/bisect.html

This module provides support for maintaining a list in sorted order without having to sort the list after each insertion.

The module is called bisect because it uses a basic bisection algorithm to do its work.

## 10.3 Hash Tables

If we put merge sort together with binary search, we have a nice way to search lists.

we can still ask, is logarithmic **the best** that we can do for  search when we are willing to do some

<font color="blue">Preprocessing</font>

###  When we introduced the type <font color="blue">dict</font> in Chapter 5 

* dictionaries use a technique called <b>hashing</b> to do <b>the lookup in time</b> 

  that is <b>nearly independent of the size of the dictionary</b>.

In [ ]:
import timeit

query_lst = [-600,-60,-6,-70,-6,0,6,70,6,60,600]

lst = []
dic = {}

# search space :lst,dic
size=10000
for i in range(size):
    lst.append(i)
    dic[i] = i 

ls="""
for v in query_lst:
    if v in lst:
        continue
"""
   
t_querylst=timeit.Timer(stmt=ls,globals=globals())
lst_time=t_querylst.timeit(10)
print('Linear search time : ',lst_time/10)

bls="""
for v in query_lst:
    if search(lst, v):
       continue
"""
t_querylst=timeit.Timer(stmt=bls,globals=globals())
bst_time=t_querylst.timeit(10)
print('Binary search time : ',bst_time/10)

ds="""
for v in query_lst:
    if v in dic:
        continue
"""
t_querydic=timeit.Timer(stmt=ds,globals=globals())
dict_time=t_querydic.timeit(10)
print('dict search time : ',dict_time/10)

### A hash table 

The basic idea behind **a hash table** is

* **convert the key to an integer, and then use that integer to index into a list**

  which can be done in constant time.

  In principle, values of any immutable type can be easily converted to an integer through **Hash functions**.


### Hash functions

**Hash functions** can be used to convert 

* <b>a large space of keys</b>  to  <b>a smaller space of integer indices</b>.

Since the space of possible outputs is**smaller** than the space of possible inputs, 

a hash function is a  <b>many-to-one mapping</b>,

multiple different inputs may be mapped to the same output. 

When two inputs are mapped to the same output, 

it is called a <b>collision</b>. 

A good hash function produces 

<b>a uniform distribution</b>

every output in the range is equally probable, which <b>minimizes the probability of collisions</b>.

**Designing good hash functions** is surprisingly challenging.

The problem is that one wants the outputs to be uniformly distributed given the expected distribution of inputs

## hash bucket

**class intDict(object)**  uses a simple hash function 

```python
# i%j returns the remainder when the integer i is divided by the integer j 

hashBucket = self.buckets[dictKey%self.numBuckets]
``` 

to implement a dictionary with integers as keys.

The basic idea is to represent 

**an instance of class intDict**

by a list of <b>hash buckets</b>, 

where **each bucket** is a list of **key/value** pairs. 

<b>By making each bucket a list, 

we handle collisions by storing all of the values that hash to the same bucket in the list</b>. 

The <b>hash table</b> works as follows: 

*  **1 def __init__(self, numBuckets):**

   The instance variable <b>buckets</b> is initialized to 

   <b>a list</b> of  <b>numBuckets</b> <b>empty lists</b>.

```python
        self.buckets = []
        self.numBuckets = numBuckets
        for i in range(numBuckets):
            self.buckets.append([]) 
```
   
* **2 def addEntry(self, dictKey, dictVal):**

   To store or look up an entry with key **dictKey**, 

```python   
        hashBucket = self.buckets[dictKey%self.numBuckets]
        for i in range(len(hashBucket)):
            if hashBucket[i][0] == dictKey:
                hashBucket[i] = (dictKey, dictVal) #if one was found,replace
                return
        hashBucket.append((dictKey, dictVal)) # append a new entry to the bucket if none was found.
```      
   
we use <b>the hash function i%j to convert dictKey into an integer</b>, 
```python  
    hashBucket = self.buckets[dictKey%self.numBuckets
```    
and use that integer to index into buckets 
```python
   hashBucket[i]
```
to find the hash bucket associated with **dictKey**

If <b>a value is to be stored</b>,then  

if one was found:  <b>replace</b> the value in the existing entry, if one was found, 

if none was found: <b>append</b> a new entry to the bucket

### separate chaining(分离链接法)

In the method known as separate chaining, each bucket is independent, and has some sort of list of entries with the same index. 

![](./img/Hashcollisionbyseparatechaining.jpg)

* **3 def getValue(self, dictKey):**

We then search that bucket (which is a list) linearly to see if there is an entry with the key dictKey.

```python 
 for e in hashBucket:
            if e[0] == dictKey: // key
                return e[1]     // valu
```

If we are doing <b>a lookup</b> and there is an entry with the key, we simply return the value stored with that key. 

If there is no entry with that key, we return None. 




In [2]:
#Page 139, Figure 10.6

class intDict(object):
    """A dictionary with integer keys"""
    
    def __init__(self, numBuckets):
        """Create an empty dictionary"""
        
        ## buckets is initialized to a list of numBuckets empty lists.
        
        self.buckets = []
        self.numBuckets = numBuckets
        for i in range(numBuckets):
            self.buckets.append([]) # empty list
            
    def addEntry(self, dictKey, dictVal):
        """Assumes dictKey an int.  Adds an entry."""
        hashBucket = self.buckets[dictKey%self.numBuckets]
        for i in range(len(hashBucket)):
            if hashBucket[i][0] == dictKey:
                hashBucket[i] = (dictKey, dictVal) #if one was found,replace
                return
        hashBucket.append((dictKey, dictVal)) # append a new entry to the bucket if none was found.
        
    def getValue(self, dictKey):
        """Assumes dictKey an int.  Returns entry associated
           with the key dictKey"""
        hashBucket = self.buckets[dictKey%self.numBuckets]
        for e in hashBucket:
            if e[0] == dictKey: # key
                return e[1]     # value 
        return None
    
    def __str__(self):
        result = '{'
        for b in self.buckets:
            for e in b:
                result = result + str(e[0]) + ':' + str(e[1]) + ','
        return result[:-1] + '}' #result[:-1] omits the last comma



The following code first constructs an **intDict** with twenty entries. 

The values of the entries are the integers 0 to 19.

The **keys** are chosen at random from <b>integers in the range 0 to 10^5 - 1<b>.

## Twenty entries

In [3]:
import random #a standard library module
Entrys=[]
NumEntry=20

random.seed(1)
for i in range(NumEntry):
    #choose a random int between 0 and 10**5
    key = random.randint(0, 10**5)
    Entrys.append((key,i))

print('The Entrys (key,i)is:')
for entry in Entrys:
    print(entry)

The Entrys (key,i)is:
(17611, 0)
(74606, 1)
(8271, 2)
(33432, 3)
(15455, 4)
(64937, 5)
(99740, 6)
(58915, 7)
(61898, 8)
(85405, 9)
(49756, 10)
(27519, 11)
(12302, 12)
(63944, 13)
(3715, 14)
(51093, 15)
(56723, 16)
(79618, 17)
(99913, 18)
(276, 19)


### Search key->value in list of Entrys

In [4]:
def searchEntrys():
    for entry in Entrys:
        key=entry[0]
        
        for item in Entrys:
            if (key==item[0]):
                value=item[1] 
                print(key,value)  
                break
                
searchEntrys()

17611 0
74606 1
8271 2
33432 3
15455 4
64937 5
99740 6
58915 7
61898 8
85405 9
49756 10
27519 11
12302 12
63944 13
3715 14
51093 15
56723 16
79618 17
99913 18
276 19


## Put entrys into intDict

* numBuckets =29

In [ ]:
numBuckets =29

D = intDict(numBuckets) # numBuckets >entries <<< the range of key （0，100000）
for item in Entrys:
    D.addEntry(item[0],item[1])

print('The intDict is:' )
print(D)

print('\n', 'The hase buckets are:')
i=0
for hashBucket in D.buckets:
    print('BucketID',i,'  ', hashBucket)
    i=i+1

we see that many of the hash buckets are **empty**. 

Others contain **one, two, or three tuples** depending upon <b>the number of collisions</b> that occurred

In [ ]:
def getIntDict():
    for entry in Entrys:
        key=entry[0]
        value=D.getValue(key)
        #print(key,value)
getIntDict()        

In [ ]:
%timeit getIntDict()

### What is the complexity of **getValue**? 

If there were <b>no collisions</b> it would be <b>O(1)</b>,  because each hash bucket would be of length 0 or 1.

there might be <b>collisions</b>. 

If everything hashed to the same bucket,

it would be <b>O(n)</b> where n is the number of entries in the dictionary,

because the code would perform a linear search on that hash bucket.

### By making the <b>hash table large enough</b>, 

we can <b>reduce the number of collisions</b> sufficiently to allow us to treat <b>the complexity as O(1)</b>.

### hash table small , more collisions

* numBucket=5

In [ ]:
import random #a standard library module

# hash table small , more collisions
numBuckets=5
D = intDict(numBuckets) # numBuckets < entries

for item in Entrys:
    D.addEntry(item[0],item[1])

print('The value of the intDict is:')
print(D)

print('\n', 'The buckets are:')
for hashBucket in D.buckets: #violates abstraction barrier
    print('  ', hashBucket)


In [ ]:
%timeit getIntDict()

### hash table large , less collisions

* numBucket=50

In [ ]:
import random #a standard library module

# hash table large , less collisions
numBucket=50

D = intDict(numBucket) # numBuckets >> entries

for item in Entrys:
    D.addEntry(item[0],item[1])
    
print('The value of the intDict is:')
print(D)

print('\n', 'The buckets are:')
for hashBucket in D.buckets: #violates abstraction barrier
    print('  ', hashBucket)

In [ ]:
%timeit getIntDict()

##  Note: 

### hash table: a series of buckets

* **bucket** : a list of key/value pairs. storing all of the values that hash to the same bucket in the list 

* **hash each element to bucket**: an address that is derived form the key value by applying the hash function
    
*  **search:**

    * key -> the hash function -> an address of bucket in the hash table -> get valve in bucket

## intDict in C

In [1]:
%%file ./code/ds/intDict.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

typedef struct _node
{
	int key;
	int value;
	struct _node *next;
} Node;

typedef struct _hashtable
{
	Node **buckets;
} Hashtable;

int inthash(int k,int  numBuckets)
{

    return k % numBuckets;
}

Node *searchEntry(Hashtable *HT, int k,int  numBuckets)
{
	Node *p;
	int addr = inthash(k, numBuckets);
	p = HT->buckets[addr];

	while(p && p->key !=k)
		p = p->next;

	return p;
}

void addEntry(Hashtable *HT, Node *entry,int numBuckets)
{
	int addr;
	Node *p;
	p = searchEntry(HT,entry->key,numBuckets);
	if(p)
	{
		free(entry);
	}
	else
	{
		addr =  inthash(entry->key, numBuckets);
		entry->next = HT->buckets[addr];
		HT->buckets[addr] = entry;
	}
}

int getValue(Hashtable *HT, int key,int numBuckets)
{   
	Node *p;
	p = searchEntry(HT,key,numBuckets);
	if (p)
	{
      return p->value;
	}
}

int main()
{
	int numBuckets=29;
    int numEntries=20;
    Hashtable *HT;
	Node **Entry;
	
    HT=(Hashtable*)malloc(sizeof(Hashtable*));
    HT->buckets=(Node**)malloc(sizeof(Node)*numBuckets);

	Entry=(Node**)malloc(sizeof(Node*)*20);

	for(int i=0;i<numBuckets;i++)
		HT->buckets[i] = NULL;
    
    printf("The value of the intDict is:\n");
    printf("(key value)\n");
    srand(time(NULL));
	for(int i=0;i<numEntries;i++)
	{
		Entry[i]=(Node*)malloc(sizeof(Node));
	  	Entry[i]->key = rand() % 100000;
		Entry[i]->value=i;
		addEntry(HT,Entry[i], numBuckets);
		printf("(%d %d)\n",Entry[i]->key,Entry[i]->value);
	}

    printf("The buckets are: \n");
	for(int i=0;i<numBuckets;i++)
	{
       Node *b,*p;
	   b = HT->buckets[i];
	   printf("bucket %d :",i);
	   if (b)
	   {
	     for(p=b; p!=NULL; p=p->next)
	        printf(" (%d %d) ",p->key,p->value);
	     printf("\n"); 		
	   }
	   else
	    	printf("\n"); 	   
	}

    printf("Hash search(even):\n");
    printf("(key value) : key -> value:\n");
	for(int i=0;i< numEntries;i++)
	{
      if (i%2==0)
      {
        int value=getValue(HT, Entry[i]->key,numBuckets);
        printf("(%d  %d): -> %d \n",Entry[i]->key,Entry[i]->value,value);
      } 
	}
    
    for(int i=0;i<numEntries;i++)
		free(Entry[i]);
    free(Entry);
    free(HT->buckets);
    free(HT);
  	return 0;
}

Overwriting ./code/ds/intDict.c


In [2]:
!gcc -o ./code/ds/intDict ./code/ds/intDict.c

In [3]:
!.\code\ds\intDict 

The value of the intDict is:
(key value)
(26577 0)
(24495 1)
(12261 2)
(27843 3)
(28380 4)
(15077 5)
(10882 6)
(16500 7)
(28782 8)
(27263 9)
(5 10)
(3893 11)
(8591 12)
(23859 13)
(31457 14)
(30204 15)
(30314 16)
(5459 17)
(14251 18)
(29786 19)
The buckets are: 
bucket 0 :
bucket 1 :
bucket 2 :
bucket 3 : (29786 19)  (27263 9)  (27843 3) 
bucket 4 :
bucket 5 : (5 10) 
bucket 6 :
bucket 7 : (5459 17)  (8591 12)  (3893 11)  (10882 6) 
bucket 8 :
bucket 9 : (30314 16) 
bucket 10 :
bucket 11 :
bucket 12 : (14251 18) 
bucket 13 : (26577 0) 
bucket 14 : (28782 8) 
bucket 15 : (30204 15) 
bucket 16 :
bucket 17 :
bucket 18 : (28380 4) 
bucket 19 : (24495 1) 
bucket 20 :
bucket 21 : (31457 14)  (23859 13) 
bucket 22 :
bucket 23 : (12261 2) 
bucket 24 :
bucket 25 :
bucket 26 : (15077 5) 
bucket 27 :
bucket 28 : (16500 7) 
Hash search(even):
(key value) : key -> value:
(26577  0): -> 0 
(12261  2): -> 2 
(28380  4): -> 4 
(10882  6): -> 6 
(28782  8): -> 8 
(5  10): -> 10 
(8591  12): -> 12 
(3145

## Further Reading

* 严蔚敏，李冬梅，吴伟民. 数据结构（C语言版），人民邮电出版社（第2版）,2015年2月  


* Mark Allen Weiss. Data Structures and Algorithm Analysis in C


* GNU C Library: Searching and Sorting : http://www.gnu.org/software/libc/manual/html_node/Searching-and-Sorting.html


* Hash table https://en.wikipedia.org/wiki/Hash_table